In [ ]:
import ROOT
import re
ROOT.ROOT.EnableImplicitMT()
#ROOT.gErrorIgnoreLevel = ROOT.kBreak
ROOT.gStyle.SetOptStat(0)
#Nice interactive Draws 
%jsroot on
from math import *

: 

## Define Sample Names

In [2]:
sample_codes =[
#    [100010,"600,200,160",ROOT.kRed+0],
#    [100011,"600,200,140",ROOT.kRed+1],
#    [100012,"600,200,120",ROOT.kRed+2],
#    [100013,"600,200,100",ROOT.kRed+3],
#    [100014,"1000,400,360",ROOT.kBlue+0],
#    [100015,"1000,400,340",ROOT.kBlue+1],
#    [100016,"1000,400,320",ROOT.kBlue+2],
#    [100017,"1000,400,300",ROOT.kBlue+3],
#    [100018,"1500,700,660",ROOT.kGreen+0],
#    [100019,"1500,700,640",ROOT.kGreen+1],
#    [100020,"1500,700,620",ROOT.kGreen+2],
#    [100021,"1500,700,600",ROOT.kGreen+3],
[100022,"1000,400,350,500",ROOT.kRed+0],
[100023,"1000,400,350,1000",ROOT.kRed+1],
[100024,"1000,400,350,1500",ROOT.kRed+2],
[100025,"1000,400,350,2000",ROOT.kRed+3],
[100026,"1000,400,350,3000",ROOT.kRed+4],
[100027,"1000,400,350,5000",ROOT.kRed+5],
[100028,"1000,400,350,7500",ROOT.kRed+6],
[100029,"1000,400,350,10000",ROOT.kRed+7],
[100030,"1500,650,600,500",ROOT.kViolet+0],
[100031,"1500,650,600,1000",ROOT.kViolet+1],
[100032,"1500,650,600,1500",ROOT.kViolet+2],
[100033,"1500,650,600,2000",ROOT.kViolet+3],
[100034,"1500,650,600,3000",ROOT.kViolet+4],
[100035,"1500,650,600,5000",ROOT.kViolet+5],
[100036,"1500,650,600,7500",ROOT.kViolet+6],
[100037,"1500,650,600,10000",ROOT.kViolet+7],
[100038,"2500,1150,1100,500",ROOT.kCyan+0],
[100039,"2500,1150,1100,1000",ROOT.kCyan+1],
[100040,"2500,1150,1100,1500",ROOT.kCyan+2],
[100041,"2500,1150,1100,2000",ROOT.kCyan+3],
[100042,"2500,1150,1100,3000",ROOT.kCyan+4],
[100043,"2500,1150,1100,5000",ROOT.kCyan+5],
[100044,"2500,1150,1100,7500",ROOT.kCyan+6],
[100045,"2500,1150,1100,10000",ROOT.kCyan+7],
[100046,"3000,1400,1350,500",ROOT.kSpring+0],
[100047,"3000,1400,1350,1000",ROOT.kSpring+1],
[100048,"3000,1400,1350,1500",ROOT.kSpring+2],
[100049,"3000,1400,1350,2000",ROOT.kSpring+3],
[100050,"3000,1400,1350,3000",ROOT.kSpring+4],
[100051,"3000,1400,1350,5000",ROOT.kSpring+5],
[100052,"3000,1400,1350,7500",ROOT.kSpring+6],
[100053,"3000,1400,1350,10000",ROOT.kSpring+7],
[100054,"5000,2400,2350,500",ROOT.kOrange+0],
[100055,"5000,2400,2350,1000",ROOT.kOrange+1],
[100056,"5000,2400,2350,1500",ROOT.kOrange+2],
[100057,"5000,2400,2350,2000",ROOT.kOrange+3],
[100058,"5000,2400,2350,3000",ROOT.kOrange+4],
[100059,"5000,2400,2350,5000",ROOT.kOrange+5],
[100060,"5000,2400,2350,7500",ROOT.kOrange+6],
[100061,"5000,2400,2350,10000",ROOT.kOrange+7]
]

## Load Samples And Filter For Intersting Tree Columns

In [ ]:
samples = list()
cnames = list()
for sample in sample_codes:
    sample_d = ROOT.RDataFrame("analysis","/work/gbird/EDMGeneratorStudies/l1-calo-llp-validation/run/submitDir09_10_11_09_{n}/data-ANALYSIS/run.root".format(n=sample[0]))
    sample_d = sample_d.Define("llpdecayDiffRatio","(subleadLLPt-leadLLPt)/(subleadLLPt+leadLLPt)")
    cnames = [n for n in sample_d.GetColumnNames() if re.search("llp",str(n),re.IGNORECASE)] #Rough filter for interesting columns
    samples.append([sample,sample_d])
print(samples[0][1].GetColumnNames())

In [4]:
triggered_samples = list()
for sample in samples:
    triggered_sample = sample[1].Filter("std::abs(pho_llp1_eta)<2 && std::abs(pho_llp2_eta)<2","Photons within acceptance")
    triggered_sample = triggered_sample.Filter("40 < pho_llp1_pt && pho_llp1_pt < 100 && 40 < pho_llp2_pt && pho_llp2_pt < 100 ","Within (approximate) Triggerable pT")
    triggered_sample = triggered_sample.Filter("llp_del_phi>(3.1459-1)","Within Delta Phi Cut",)
    triggered_sample = triggered_sample.Filter("subleadLLPLxy < 2000 && leadLLPLxy <2000","Photons within rough radius ")
    triggered_sample = triggered_sample.Filter("pho_llp_del_t>25 && pho_llp_del_t<50","Matches timing needs")
    triggered_samples.append([sample[0],triggered_sample])

In [ ]:
sample_idx = 0
for triggered_sample in triggered_samples:
    print(triggered_sample[0])
    #report = triggered_sample[1].Report().GetValue()
    #report.Print()
    print(triggered_sample[0][1].split(",")[3])
    print(triggered_sample[1].Count().GetValue())
    sample_idx += 1

In [ ]:
# Define lifetimes and mass points
lifetimes = [500, 1000, 1500, 2000, 3000, 5000, 7500, 10000]
mass_points =  [1000,1500,2500,3000,5000]
print(len(lifetimes))

# Create a 2D TEfficiency object
efficiency = ROOT.TEfficiency("efficiency", "my efficiency;c#tau;MassIdx", len(lifetimes), 0, 11000, len(mass_points), 0, len(mass_points))
efficiency1Ds = [ROOT.TEfficiency("efficiency", f"efficiency {mass_points[idx]};c#tau;#epsilon", len(lifetimes), 0, 11000) for idx in range(len(mass_points)) ] 

# Loop over triggered samples
sample_idx = 0

for triggered_sample in triggered_samples:
    # Extract mass and lifetime from the sample description
    mass_s = int(triggered_sample[0][1].split(",")[0])  # Assuming mass is the first element
    lifetime = int(triggered_sample[0][1].split(",")[3])  # Assuming lifetime is the fourth element

    # Get total and triggered events
    total_events = samples[sample_idx][1].Count().GetValue()
    triggered_events = triggered_sample[1].Count().GetValue()
 #   print(f"Sample {sample_idx}: Mass = {mass_s}, Lifetime = {lifetime}, Total Events = {total_events}, Triggered Events = {triggered_events}")

    # Find the index of the mass point
    mass_index = mass_points.index(mass_s) if mass_s in mass_points else -1

    # Fill the efficiency object if the mass index is valid, this is a silly way to do it, oh well
    if mass_index != -1 and total_events > 0:
        for _ in range(triggered_events):
            efficiency.Fill(True, lifetime, mass_index)  # Triggered events
            efficiency1Ds[mass_index].Fill(True, lifetime)  # Triggered events
        for _ in range(total_events - triggered_events):
            efficiency.Fill(False, lifetime, mass_index)  # Non-triggered events
            efficiency1Ds[mass_index].Fill(False, lifetime )  # Non-Triggered events

    sample_idx += 1

# Create a canvas to draw the efficiency plot
c1 = ROOT.TCanvas("c1", "Efficiency vs Lifetime", 800, 600)

# Draw the efficiency plot
efficiency.Draw("COLZ")  # Use COLZ for a 2D color plot
c1.Draw()

# Save the canvas
c1.SaveAs("Images/efficiency_vs_lifetime_2D.png")

# Keep the canvas in scope
c1.Update()  # Refresh the canvas to ensure it displays correctly

In [ ]:
c_effproject = ROOT.TCanvas("Distributions","Distibutions",1000,2000)
c_effproject.Divide(2,ceil(len(mass_points)/2))
c_idx = 1
for p in efficiency1Ds:
    c_effproject.cd(c_idx)
    p.Draw("COLZ")
    c_idx+=1
c_effproject.Draw()

In [ ]:
first_sample = True 
hists = [] #block garbage collection
cdists = ROOT.TCanvas("Distributions","Distibutions",1500,7000)
cdists.Divide(2,ceil(len(cnames)/2))
l = ROOT.TLegend()
projections = []
models = dict()
max_values = dict()
first_sample = True
for label, d in samples:
   # d = base_d.Filter("subleadLLPboost<0.7")
    colidx=1
    first_column = True
    for name in cnames:
        cdists.cd(colidx)
        #ROOT.gPad.SetLogy() 
        if first_sample:
            hists.append(d.Histo1D(name))
            model = ROOT.RDF.TH1DModel(hists[-1].GetValue())
            model.fXUp = ceil(model.fXUp /2)
            if name == "pho_llp_del_t" or name == "subleadLLPt" or name == "leadLLPt":
                model.fXUp = 100 #Some issue with finding min value if too small
            if model.fXLow > 0 : model.fXLow = 0
            model.fNbinsX = ceil(model.fNbinsX /4)
            models[name] = model 
            max_values[name] =  1.2*hists[-1].GetMaximum()
            hists[-1]= d.Histo1D(model,name)
        else:
            hists.append(d.Histo1D(models[name],name))
        #hists[-1].Rebin(2) # Must be before y-axis rescaling
        hists[-1].SetMaximum(2*hists[-1].GetMaximum())
        hists[-1].SetLineColor(label[2])
        if first_column:
            l.AddEntry(hists[-1].GetValue(),label[1],"l")
           # ROOT.TH1DModel(hists[-1])
        hists[-1].Draw("SAME")
        colidx+=1
        first_column = False
    first_sample = False
    projections.append(d.Histo2D((label[1],"m_{s},m_{1},m_{0}"+label[1],20,0,1,20,0,1),"leadLLPboost","subleadLLPboost"))
for canvas_index in range(len(cnames)):
    cdists.cd(canvas_index+1)
    l.SetHeader("m_{s},m_{1},m_{0}")
    l.Draw()
cdists.Draw()


In [ ]:
c_2Ddists = ROOT.TCanvas("Distributions","Distibutions",1000,3000)
c_2Ddists.Divide(2,ceil(len(projections)/2))
c_idx = 1
for p in projections:
    c_2Ddists.cd(c_idx)
    p.Draw("COLZ")
    c_idx+=1
c_2Ddists.Draw()

In [ ]:
first_sample = True 
hists = [] #block garbage collection
cdists = ROOT.TCanvas("Distributions","Distibutions",1500,7000)
cdists.Divide(2,ceil(len(cnames)/2))
l = ROOT.TLegend()
projections = []
models = dict()
max_values = dict()
first_sample = True
for label, d in triggered_samples:
   # d = base_d.Filter("subleadLLPboost<0.7")
    colidx=1
    first_column = True
    for name in cnames:
        cdists.cd(colidx)
        #ROOT.gPad.SetLogy() 
        if first_sample:
            hists.append(d.Histo1D(name))
            model = ROOT.RDF.TH1DModel(hists[-1].GetValue())
            model.fXUp = ceil(model.fXUp /2)
            if name == "pho_llp_del_t" or name == "subleadLLPt" or name == "leadLLPt":
                model.fXUp = 100 #Some issue with finding min value if too small
            if model.fXLow > 0 : model.fXLow = 0
            model.fNbinsX = ceil(model.fNbinsX /4)
            models[name] = model 
            max_values[name] =  1.2*hists[-1].GetMaximum()
            hists[-1]= d.Histo1D(model,name)
        else:
            hists.append(d.Histo1D(models[name],name))
        #hists[-1].Rebin(2) # Must be before y-axis rescaling
        hists[-1].SetMaximum(2*hists[-1].GetMaximum())
        hists[-1].SetLineColor(label[2])
        if first_column:
            l.AddEntry(hists[-1].GetValue(),label[1],"l")
           # ROOT.TH1DModel(hists[-1])
        hists[-1].Draw("SAME")
        colidx+=1
        first_column = False
    first_sample = False
    projections.append(d.Histo2D((label[1],"m_{s},m_{1},m_{0}"+label[1],20,0,1,20,0,1),"leadLLPboost","subleadLLPboost"))
for canvas_index in range(len(cnames)):
    cdists.cd(canvas_index+1)
    l.SetHeader("m_{s},m_{1},m_{0}")
    l.Draw()
cdists.Draw()


In [ ]:
c_2Ddists = ROOT.TCanvas("Distributions","Distibutions",1000,3000)
c_2Ddists.Divide(2,ceil(len(projections)/2))
c_idx = 1
for p in projections:
    c_2Ddists.cd(c_idx)
    p.Draw("COLZ")
    c_idx+=1
c_2Ddists.Draw()